Ceci est le code pour le projet de green ai ayant pour but de faire un dashboard intéractif permettant de visualiser le scoring de chaque centrale nucléaire et de chaque région et prédire leur émission de CO2. On prendra en compte dans le même temps les besoin énergétique de chaque région et poseront différentes équations permettant de mieux analysé chaque centrale.
Le dataset sera issus d'une source vérifié qui data.gouv et du site de edf : https://opendata.edf.fr/explore/?sort=modified&disjunctive.theme&disjunctive.publisher&disjunctive.keyword. Cette solution à pour but de promouvoir la transition énergétique des différentes régions et de pouvoir poser un état des lieux de la situation national et international.
Notre solution sera basé sur un volet d'analyse avec jupyter notebook et enfin d'une web app permettant de visualisé et comprendre les différents résultat d'analyse.

In [73]:
import pandas as pd
import folium

datacentrale = pd.read_excel("Centrale data.gouv//Excel centrales.xlsx")
CO2pays = pd.read_csv("Centrale data.gouv//emissions-de-co2-consolidees-par-pays-du-groupe-edf.csv",encoding="utf-8",sep=";")
datacentrale.columns = [col.lower() for col in datacentrale.columns]
datacentrale[['latitude', 'longitude']] = datacentrale['point_gps_wsg_84'].str.split(',', expand=True)
datacentrale['latitude'] = datacentrale['latitude'].astype(float)
datacentrale['longitude'] = datacentrale['longitude'].astype(float)

Affichage Dataframe

In [55]:
"""print(eolinne.columns)
print(hydraulique.columns)
print(nucléaire.columns)
print(thermique.columns)
print(CO2pays.columns)"""
print(len(datacentrale))
print(datacentrale)
print(len(CO2pays))


183
     tri perimetre_juridique  \
0      1              EDF SA   
1      2              EDF SA   
2      3              EDF SA   
3      4              EDF SA   
4      5              EDF SA   
..   ...                 ...   
178   56              EDF SA   
179   57              EDF SA   
180   74              EDF SA   
181   75              EDF SA   
182   76              EDF SA   

                                     perimetre_spatial  \
0    France mÃ©tropolitaine, sans la Corse ni les i...   
1    France mÃ©tropolitaine, sans la Corse ni les i...   
2    France mÃ©tropolitaine, sans la Corse ni les i...   
3    France mÃ©tropolitaine, sans la Corse ni les i...   
4    France mÃ©tropolitaine, sans la Corse ni les i...   
..                                                 ...   
178  France métropolitaine, sans la Corse ni les il...   
179  France métropolitaine, sans la Corse ni les il...   
180  France métropolitaine, sans la Corse ni les il...   
181  France métropolitaine, san

Calcul des différents taux d'émissions de CO2

In [74]:
emission_factors = {
    'Hydraulic': 4,
    'Nuclear': 12,
    'Coal': 820,
    'Gas': 490,
    'Oil': 650,
    'Renewable': 10  # Placeholder for all renewables
}

# Assume a utilization factor for each plant type to estimate annual production
utilization_factors = {
    'Hydraulic': 0.5,
    'Nuclear': 0.9,
    'Coal': 0.7,
    'Gas': 0.6,
    'Oil': 0.6,
    'Renewable': 0.3
}

# Function to estimate annual production in kWh
def estimate_annual_production(capacity_mw, sector):
    # Convert capacity to kWh and multiply by utilization factor and hours per year
    return capacity_mw * 1e6 * utilization_factors.get(sector, 0.5) * 24 * 365

# Function to calculate CO2 emissions based on sector and installed capacity
def calculate_co2_emissions(sector, capacity_mw):
    annual_production_kwh = estimate_annual_production(capacity_mw, sector)
    # Calculate emissions in metric tons (1 metric ton = 1e6 grams)
    return annual_production_kwh * emission_factors.get(sector, 10) / 1e6

# Calculate CO2 emissions for each row in the dataframe
datacentrale['co2_emissions_ton'] = datacentrale.apply(lambda x: calculate_co2_emissions(x['sector'], x['puissance_installee']), axis=1)
print(datacentrale["co2_emissions_ton"])

0       6307200.0
1       1822080.0
2      10547040.0
3       4117200.0
4       7113120.0
          ...    
178     8103000.0
179     8103000.0
180     7927800.0
181     7840200.0
182     7971600.0
Name: co2_emissions_ton, Length: 183, dtype: float64


Génération de la carte intéractive des différentes centrales

In [76]:
def get_marker_color(sector, co2_emissions):
    colors = {
        'Hydraulic': 'green',
        'Nuclear': 'blue',
        'Coal': 'darkred',
        'Gas': 'lightred',
        'Oil': 'red',
        'Renewable': 'lightgreen',
        # Add other sectors as needed
    }
    # Adjust the color based on CO2 emissions, if necessary
    # For example, you might have a logic to adjust the shade based on emissions
    return colors.get(sector, 'gray')


m = folium.Map(location=[46.2276, 2.2137], zoom_start=6)  # Centered on France

# Add points to the map
for idx, row in datacentrale.iterrows():
    try:
        # Ensure that latitude and longitude are not NaN before adding them to the map
        if not pd.isna(row['latitude']) and not pd.isna(row['longitude']):
            marker_color = get_marker_color(row['sector'], row['co2_emissions_ton'])
            # Create the popup message
            popup_message = folium.Popup(
                f"Tri: {row['tri']}<br>"
                f"Sector: {row['sector']}<br>"
                f"Power Plant: {row['centrale']}<br>"
                f"Power Station Category: {row['power_station_category']}<br>"
                f"Year of Service: {row['annee_de_mise_en_service']}<br>"
                f"Installed Capacity: {row['puissance_installee']} {row['unite']}<br>"
                f"Department: {row['departement']}<br>"
                f"Postal Code: {row['code_postal']}<br>"
                f"Commune: {row['commune']}<br>"
                f"CO2 Emissions (tons/year): {row['co2_emissions_ton']}",
                max_width=300
            )
            # Add marker to the map with the popup
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=popup_message,
                icon=folium.Icon(color=marker_color)
            ).add_to(m)
    except ValueError as e:
        print(f"Error adding marker for row {idx}: {e}")

# Display the map
m.save('power_plants_map.html')
